In [768]:
import urllib
import cssselect
from lxml.html import fromstring
from urllib import urlopen
from lxml.etree import XMLSyntaxError
import csv
import pymorphy2 as pym
import nltk
from itertools import groupby
import math
import operator
from pymystem3 import Mystem
from collections import defaultdict
import chardet 
from tqdm import tqdm_notebook
from difflib import get_close_matches

In [930]:
#constants
COUNTER = 10

URL_rbk = 'https://www.rbc.ru/politics'
ITEM_PATH_rbk = '.l-row .item .item__link'
IN_ITEM_PATH_rbk = '.article__content .article__text'
HEADER_rbk = '.l-row .article__header__title .js-slide-title'

URL_mail = 'https://news.mail.ru/politics'
ITEM_PATH_mail = '.cols__inner .grid__fixer .photo'
IN_ITEM_PATH_mail = '.article__item .article__item_alignment_left .article__item_html'

URL_rns = 'https://rns.online/society/'
ITEM_PATH_rns = '.b-list__container .b-list__time-slice .b-list__items'
IN_ITEM_PATH_rns = '.b-news .t-body'
IN_ITEM_TIME_rns = '.b-news .b-news__rubric-p .b-news__time'

URL_vedomosti = 'https://www.vedomosti.ru/newsline/politics'
ITEM_PATH_vedomosti = '.b-rubric_news .newsline_item .b-newsline-item__title'
IN_ITEM_PATH_vedomosti ='.b-news_wrapper-first .b-news-item__wrapper .b-news-item__text'
HEADER_vedomosti = '.header-h0'
IN_ITEM_TIME_vedomosti = '.b-news-item__wrapper .b-news-item__title b-news-item__title_one .b-news-item_one__params .pubdate .b-published_at_time'


In [931]:
rbk_row_data = dict()
header_row_rbk = dict()
header_rbk = dict()
rbk_data = dict()
rns_row_data = dict()
header_row_rns = dict()
header_rns = dict()
rns_data = dict()
used = list()
vedomosti_row_data = dict()
vedomosti_data = dict()
header_row_vedomosti = dict()
header_vedomosti = dict()
data = dict()
tickers_data = dict()
tickers = list()
sites = {'rbk', 'rns', 'vedomosti'}
class News:
    """text, header, tickers, site, time, score, score_tickers"""
    def __init__(self, text, header, tickers, site, time, score=0, score_tickers=0, count=0):
        self.text = text
        self.header = header
        self.tickers = tickers
        self.site = site
        self.time = time
        self.score = score
        self.score_tickers = score_tickers
        self.count = count

In [932]:
b = 'власть власти властью властей властям властями властях путин путина путину путине путиным общество общества обществу обществе обществом украина украине украиной украины граница границы границей границе границы границу партия партии партией партиями партию партий партиям партиях запад запада западу западом западе депутат депутаты депутата депутатов депутату депутатам депутатом депутатами депутате депутатах государство государства государству государством государстве государств государствам государствами государствах коррупция коррупции коррупцию коррупции коррупцией политик политика политику политиком политике трамп трампа трампу трампом трампе саммит саммита саммиту саммитом саммите санкция санкции санкцией санкциями санкций санкциям санкциях боевики боевиков боевикам боевиками боевиках ядерный ядерное ядерная ядерные ядерных ядерного ядерной ядерным ядерному ядерными ядерном ядерными миротворцы миротворец миротворца миротворцев миротворцу миротворцам миротворцем миротворцами миротворце миротворцах оппозиция оппозиции оппозицией выборы выборов выборам выборами выборах'
a = 'Власть Власти Властью Властей Властям Властями Властях Путин Путина Путину Путине Путиным Общество Общества Обществу Обществе Обществом Украина Украине Украиной Украины Граница Границы Границей Границе Границы Границу Партия Партии Партией Партиями Партию Партий Партиям Партиях Запад Запада Западу Западом Западе Депутат Депутаты Депутата Депутатов Депутату Депутатам Депутатом Депутатами Депутате Депутатах Государство Государства Государству Государством Государстве Государств Государствам Государствами Государствах Коррупция Коррупции Коррупцию Коррупции Коррупцией Политик Политика Политику Политиком Политике Трамп Трампа Трампу Трампом Трампе Саммит Саммита Саммиту Саммитом Саммите Санкция Санкции Санкцией Санкциями Санкций Санкциям Санкциях Боевики Боевиков Боевикам Боевиками Боевиках Ядерный Ядерное Ядерная Ядерные Ядерных Ядерного Ядерной Ядерным Ядерному Ядерными Ядерном Ядерными Миротворцы Миротворец Миротворца Миротворцев Миротворцу Миротворцам Миротворцем Миротворцами Миротворце Миротворцах Оппозиция Оппозиции Оппозицией Выборы Выборов Выборам Выборами Выборах'
company = b.split(' ')
compa = a.split(' ')
company.extend(compa[:])

In [933]:
def check_for_tickers(text, key):
    tickers = ''
    wordes = list()
    paper = ' '
    words = text.split(',')
    b = ' '.join(words)
    wordes = b.split()
    for word in wordes:
        if word[-1:] == ')':
            word = word[:-1]
        if word[:1] == '(':
            word = word[1:]
        if word[-1:] == '?':
            word = word[:-1]
        if word[-1:] == ']':
            word = word[:-1]
        if word[:1] == '[':
            word = word[1:]
        if word[:2] == '«':
            word = word[2:]
        if word[-2:] == '»':
            word = word[:-2]
        if word[:1] == '"':
            word = word[1:]
        if word[-1:] == '"':
            word = word[:-1]
        if word[:1] == "'":
            word = word[1:]
        if word[-1:] == "'":
            word = word[:-1]
        if word[:1] == "'":
            word = word[1:]
        if word[-1:] == "'":
            word = word[:-1]
        paper += ' ' + word
    for ticker in company:
        if ticker in paper:
            tickers+=' '+ ticker
    if tickers == '':
        return 'nothing'
    else:
        return tickers
    

In [934]:
def clear():
    rbk_data.clear()
    rbk_row_data.clear()
    used[:]=[]
    vedomosti_data.clear()
    vedomosti_row_data.clear()
    rns_data.clear()
    rns_row_data.clear()
    header_rns.clear()
    header_row_rns.clear()
    header_rbk.clear()
    header_row_rbk.clear()
    header_vedomosti.clear()
    header_row_vedomosti.clear()
    data.clear()
clear()

In [935]:
def parse():
    parse_news_rbk()
    parse_news_rns()
    parse_news_vedomosti()
    export()

In [936]:
def export():
    with open('database.csv', 'w') as csv_file:
        writer = csv.writer(csv_file)
        for key, n in data.items():
            writer.writerow([n.site , key, n.text, n.header, n.tickers, n.time, n.score, n.score_tickers, n.count])        

In [937]:
def import_from_csv():
    with open('database.csv', 'r') as csv_file:
        reader = csv.reader(csv_file)
        for row in reader:
            try:
                n = News(row[2], row[3], row[4], row[0], row[5], row[6], row[7])
                data.update({row[1]:n})
            except IndexError:
                continue

In [938]:
clear()
export()

In [939]:
def parse_news_rns():
    timing = dict()
    timing.clear()
    f = urlopen(URL_rns)
    list_html = f.read().decode('utf-8')
    list_doc = fromstring(list_html)
    counter = 0
        
    for elem in tqdm_notebook(list_doc.cssselect(ITEM_PATH_rns)):
        if counter > COUNTER:
            break
        
        counter+=1
        a = elem.cssselect('a')[0]
        header = a.text
        href = a.get('href')
        if 'http' in href:
            continue
        real_href = 'https://rns.online' + href
        header_row_rns.update({real_href:header})
        information = urlopen(real_href).read().decode('utf-8')
        try:
            news = fromstring(information)
        except XMLSyntaxError:
            continue
        my_time = news.cssselect(IN_ITEM_TIME_rns)
        descr_news = news.cssselect(IN_ITEM_PATH_rns)
        text = list()
        
        
        times = my_time[0].cssselect('time')
        tim = times[0].text
        time = tim.split(',')
        time = time[1]
        time = time[1:]
            
        timing.update({real_href:time})
        for part in tqdm_notebook(descr_news):
            for i in range(10):
                try:
                    
                    p = part.cssselect('p')[i]
                    row = p.text_content()
                    if row != None:
                        text.append(row)
                except IndexError:
                    continue
            rns_row_data.update({real_href:text})
        
    a = str()
    b = str() 
    for key in header_row_rns.keys():
        b = ''
        x = header_row_rns[key]
        for string in x:
            b += string.encode('utf-8')
        header_rns[key] = b
        
    for key in rns_row_data.keys():
        a = ''
        x = rns_row_data[key]

        for string in x:
            a += ' ' + string.encode('utf-8')
        if not key in data.keys():
            main_time = timing[key]
            n = News(a, header_rns[key], check_for_tickers(a, key), 'rns', timing[key])
            data[key] = n
        if data[key].tickers == 'nothing':
            data.pop(key)
            rns_row_data.pop(key)

In [940]:
def parse_news_vedomosti():
    timing_vedomosti = dict()
    timing_vedomosti.clear()
    f = urlopen(URL_vedomosti)
    list_html = f.read().decode('utf-8')
    list_doc = fromstring(list_html)
    counter = 0
    for elem in tqdm_notebook(list_doc.cssselect(ITEM_PATH_vedomosti)):
        if counter > COUNTER:
            break
        counter+=1
        
        my_time = elem.cssselect('.b-newsline-item__title')   
        times = my_time[0].cssselect('time')
        time = times[0].text
        a = elem.cssselect('a')[0]
        header = a.text
        href = a.get('href')
        if 'http' in href:
            continue
        real_href = 'https://www.vedomosti.ru' + href
        timing_vedomosti.update({real_href:time})
        header_row_vedomosti.update({real_href:header})
        information = urlopen(real_href).read().decode('utf-8')
        try:
            news = fromstring(information)
        except XMLSyntaxError:
            continue
        descr_news = news.cssselect(IN_ITEM_PATH_vedomosti)
        text = list()

        for part in tqdm_notebook(descr_news):
            for i in range(5):
                try:
                    p = part.cssselect('p')[i]
                    row = p.text_content()
                    if row != None:
                        text.append(row)
                except IndexError:
                    continue
            break
        vedomosti_row_data.update({real_href:text})
    a = str()
    b = str() 
    for key in header_row_vedomosti.keys():
        b = ''
        x = header_row_vedomosti[key]

        for string in x:
            b += string.encode('utf-8')
        header_vedomosti[key] = b
        
    for key in vedomosti_row_data.keys():
        a = ''
        x = vedomosti_row_data[key]
        for string in x:
            a += ' ' + string.encode('utf-8')
        if not key in data.keys():
            main_time = timing_vedomosti[key]
            n = News(a, header_vedomosti[key], check_for_tickers(a, key), 'vedomosti', main_time)
            data[key] = n
        if data[key].tickers == 'nothing':
            data.pop(key)
            vedomosti_row_data.pop(key)

In [941]:
def parse_news_rbk():
    timing = list()
    timing = []
    order = list()
    order = []
    ordered_time = dict()
    ordered_time.clear()
    f = urlopen(URL_rbk)
    list_html = f.read().decode('utf-8')
    list_doc = fromstring(list_html)

    
    
    counter = 0 
    
    
    for elem in tqdm_notebook(list_doc.cssselect('.l-row .item .item__bottom .item__info')):
        if counter > COUNTER:
            break
        counter+=1
        times = elem.cssselect('span')
        time = times[0].text.encode('utf-8')
        timing.append(time)
    
    
    counter = 0 
    
    
    for elem in tqdm_notebook(list_doc.cssselect(ITEM_PATH_rbk)):
        if counter > COUNTER:
            break
        counter+=1
        
        a = elem.cssselect('a')[0]
        href = a.get('href')
        order.append(href)
        information = urlopen(href).read().decode('utf-8')
        try:
            news = fromstring(information)
        except XMLSyntaxError:
            continue
        h = news.cssselect(HEADER_rbk)[0]
        header = h.text_content()
        header_row_rbk.update({href:header})
        descr_news = news.cssselect(IN_ITEM_PATH_rbk)
        text = list()
        
        for part in descr_news:
            for i in range(10):
                try:
                    p = part.cssselect('p')[i]
                    row = p.text_content()
                    if row != None:
                        text.append(row)
                except IndexError:
                    continue
        rbk_row_data.update({href:text})
        
    p = len(timing)
    for i in range(p):
        t = timing.pop()
        o = order.pop()
        ordered_time[o] = t
    
    a = str()
    b = str() 
    
    
    for key in header_row_rbk.keys():
        b = ''
        x = header_row_rbk[key]
        for string in x:
            b += string.encode('utf-8')
        header_rbk[key] = b

    for key in rbk_row_data.keys():
        a = ''
        x = rbk_row_data[key]

        for string in x:
            a += ' ' + string.encode('utf-8')
        if not key in data.keys():
            n = News(a, header_rbk[key], check_for_tickers(a, key), 'rbk', ordered_time[key])
            data[key] = n
        if data[key].tickers == 'nothing':
            data.pop(key)
            rbk_row_data.pop(key) 

In [942]:
clear()
import_from_csv()
parse()

10:39
https://www.rbc.ru/rbcfreenews/5b5c1d589a79474a6ff2b162
27 июл, 23:25
https://www.rbc.ru/politics/27/07/2018/5b5b7e8b9a79472bc5c170f0
12:48
https://www.rbc.ru/rbcfreenews/5b5c370d9a79474de602c754
11:59
https://www.rbc.ru/politics/28/07/2018/5b5c304c9a79474d32542cca
27 июл, 22:44
https://www.rbc.ru/rbcfreenews/5b5b72779a794728e99830ac
27 июл, 23:11
https://www.rbc.ru/politics/27/07/2018/5b5b79949a794729c36f03ac
00:52
https://www.rbc.ru/rbcfreenews/5b5b94019a794730ea399387
08:07
https://www.rbc.ru/politics/28/07/2018/5b5bf7bc9a794742f6e038f5
08:23
https://www.rbc.ru/rbcfreenews/5b5bfc139a794743fd1d64c2
02:08
https://www.rbc.ru/rbcfreenews/5b5ba5889a7947344967766c
14:26
https://www.rbc.ru/politics/28/07/2018/5b5c4a469a794751f0c88e89
04:50
https://www.rbc.ru/rbcfreenews/5b5bcb9f9a79473a463a1deb
12:07
https://www.rbc.ru/rbcfreenews/5b5c322c9a79474d6834d991
01:21
https://www.rbc.ru/rbcfreenews/5b5b9ab99a794731470a0c70
27 июл, 22:14
https://www.rbc.ru/politics/27/07/2018/5b5b6ca29a79472

In [944]:
def normalize():
    for key in data.keys():
        dat =data[key].text
        wordes = list()
        paper = ' '
        words = dat.split(',')
        b = ' '.join(words)
        wordes = b.split()
        for word in wordes:
            if word[-1:] == ')':
                word = word[:-1]
            if word[:1] == '(':
                word = word[1:]
            if word[-1:] == ']':
                word = word[:-1]
            if word[:1] == '[':
                word = word[1:]
            if word[-1:] =='.':
                word = word[:-1]
            if word[:2] == '«':
                word = word[2:]
            if word[-2:] == '»':
                word = word[:-2]
            if word[:1] == '"':
                word = word[1:]
            if word[-1:] == '"':
                word = word[:-1]
            if word[:1] == "'":
                word = word[1:]
            if word[-1:] == "'":
                word = word[:-1]
            if word[:1] == "'":
                word = word[1:]
            if word[-1:] == "'":
                word = word[:-1]
            paper += ' ' + word
        data[key].text = paper

In [945]:
normalize()
all_data = []
le = int()
text = ''
all_dat = []
text = []
for key in data.keys():
    text = data[key].text.split(' ')
    le+=len(text)
    for word in text:
        all_dat.append(word)
        
all_dat = [e for i,e in enumerate(all_dat) if e not in all_dat[:i]]
diction = dict()
diction.clear()
for word in all_dat:
    if word!='':
        diction[word] = mystem.lemmatize(word)[0]


CPU times: user 916 ms, sys: 112 ms, total: 1.03 s
Wall time: 1.34 s


In [947]:
def normalize_syn(dat):
    new_dat = str()
    text = dat.split(' ')
    for word in text:
        try:
            word = diction[word]
        except:
            KeyError
        new_dat+=word+' '
    return new_dat

In [948]:
TF_IDF_words = defaultdict(lambda : defaultdict(set))

In [949]:
def TF(word, dat):
    text = dat.split(' ')
    in_data = text.count(word)
    r = float()
    b = in_data/float(len(text))
    return b

In [950]:
def IDF(word, site):
    D=0
    k=0
    for key in data.keys():
        if data[key].site == site:
            D+=1
            text = data[key].text
            if word in text:
                k+=1
    return math.log(D/k)

In [952]:
for key in data.keys():
    data[key].text = normalize_syn(data[key].text)


In [954]:
TF_IDF_words.clear()
def TFIDF():

    for site in sites:            
        for key in data.keys():
            if data[key].site == site:
                text = data[key].text.split(' ')
                for word in text:
                    if not TF_IDF_words[key][word]:
                        TF_IDF_words[key][word] = TF(word, data[key].text)*IDF(word, site)

In [955]:
TFIDF()
for key in TF_IDF_words:
    data[key].main_words = ''
    for i in range(5):
        a=0.0
        for word in TF_IDF_words[key]:
            if TF_IDF_words[key][word]>=a:
                a=TF_IDF_words[key][word]
                ans_word = word
                ans_key = key
        TF_IDF_words[ans_key][ans_word] = 0.0
        data[ans_key].main_words += ans_word + ' '
            


In [956]:
main = list()
coincidences = defaultdict(lambda : defaultdict(set))
coincidences.clear()
main_another = list()
main = list()
main_another = []
main = []

for key in data.keys():
    for another in data.keys():
        c=0
        if key!=another and data[key].site != data[another].site:
            a = data[key].main_words
            b = data[another].main_words
            main = a.split(' ')
            main_another = b.split(' ')
            ind=5
            for word in main:
                ind-=1
                ind_another=5
                for another_word in main_another:
                    ind_another-=1
                    if word == another_word:
                        c+=ind_another*ind
        coincidences[key][another] = c

In [957]:
coincidences_ticker = defaultdict(lambda : defaultdict(set))
coincidences_ticker.clear()
for key in coincidences.keys():
    for another in coincidences[key].keys():
        coincidences_ticker[key][another] = 0
        
        
for key in coincidences_ticker.keys():
    for another in coincidences_ticker[key].keys():
        try:
            tickers_key=data[key].tickers.split(' ')
            for tick in tickers_key:
            
                tickers_another=data[another].tickers.split(' ')
                for an_tick in tickers_another:
                    if tick == an_tick:
                        coincidences_ticker[key][another]+=1
        except:
            KeyError
        

In [958]:
score_ticker = dict()
for key in coincidences_ticker.keys():
    score_ticker[key] = 0
    
for key in coincidences_ticker.keys():
    for another in coincidences_ticker[key].keys():
        if another != key:
            score_ticker[key]+=coincidences_ticker[key][another]

In [959]:
score = dict()
for key in coincidences.keys():
    score[key] = 0
    
for key in coincidences.keys():
    for another in coincidences[key].keys():
         if another != key:
            score[key]+=coincidences[key][another]
                    

In [962]:
for key in data.keys():
    data[key].score = score[key]
    data[key].score_tickers = score_ticker[key]


In [963]:
for key in data.keys():
    data[key].count = len(data[key].text.split(' '))

In [964]:
with open('database.csv', 'r') as csv_file:
    reader = csv.reader(csv_file)
    for row in reader:
        try:
            data[row[1]].text = row[2]
        except IndexError:
            continue
export()



In [965]:
import pandas as pd

In [966]:
with open("database.csv", "r") as fp:
    reader = csv.reader(fp)

In [967]:
df = pd.read_csv('database.csv', header=-1, names=['site', 'link', 'text', 'header', 'tickers', 'time' ,'score', 'score_tickers', 'count'])

In [968]:
df

,site,link,text,header,tickers,time,score,score_tickers,count
0,vedomosti,https://www.vedomosti.ru/politics/news/2018/07...,\n Арестованная в США ...,Bloomberg сообщил о связях Бутиной с наследник...,партии Путин Путина,22:46,36,67,226
1,rbk,https://www.rbc.ru/politics/27/07/2018/5b5b7e8...,"Бывший глава Армении Роберт Кочарян, обвиняем...",Суд арестовал экс-президента Армении Роберта К...,выборах,"27 июл, 23:25",30,54,285
2,vedomosti,https://www.vedomosti.ru/politics/news/2018/07...,\n Останки военнослужа...,Белый дом сообщил о передаче Северной Кореей о...,властям властями саммит саммита Трамп Трампом,05:35,42,76,115
3,rbk,https://www.rbc.ru/rbcfreenews/5b5b72779a79472...,Президент Эквадора Ленин Морено не давал расп...,Власти Эквадора опровергли предложение выгнать...,власти,"27 июл, 22:44",30,58,172
4,rbk,https://www.rbc.ru/rbcfreenews/5b5b94019a79473...,Мэр Москвы Сергей Собянин уволил первого замр...,Собянин уволил замглавы департамента экономиче...,политик,00:52,30,55,123
5,rns,https://rns.online/society/SMI-Iosif-Kobzon-vp...,Сестра народного артиста СССР и депутата Госд...,Сестра Иосифа Кобзона опровергла информацию о...,депутат депутата,13:06,38,58,103
6,vedomosti,https://www.vedomosti.ru/politics/news/2018/07...,\n Премьер-министр РФ ...,"Правительство выделило 1,3 млрд рублей на моде...",общество,15:57,36,52,114
7,vedomosti,https://www.vedomosti.ru/politics/news/2018/07...,\n Национальный центр ...,Разведка США назвала Россию одним из трех лиде...,государстве государств,21:40,36,73,150
8,rbk,https://www.rbc.ru/rbcfreenews/5b5bcb9f9a79473...,"Президент США Дональд Трамп заявил, что не до...",Трамп заявил о недопустимости иностранного вме...,власти властям властями государства государст...,04:50,62,109,153
9,rns,https://rns.online/society/Zaderzhan-vosmoi-po...,По делу о пытках в ярославской колонии задерж...,Задержан восьмой подозреваемый по делу о пытка...,Обществе,11:58,38,51,110
